In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from transformers import AutoModel, BertTokenizer
device = torch.device("cpu")

In [ ]:
file = '0.npy'
item_with_index = np.load(file, allow_pickle=True)
item = item_with_index[0].astype(np.float16)
max_len = 2537
def add_padding(max_frames, item):
    current_length = item.shape[0]
    if current_length < max_frames:
        padding = np.full(
            (max_frames - current_length, 2172), 
            -1,
            dtype=np.float16
        )
        padding[:, 3::4] = 0
        item = np.concatenate((item, padding), axis=0)
    return item
matrix = add_padding(max_len, item)
print(matrix.shape)

## PCA

In [3]:
# Paso 1: Generar la matriz y centrar los datos restando la media de cada columna
matrix_meaned = matrix - np.mean(matrix, axis=0)

# Paso 2: Calcular la matriz de covarianza
cov_matrix = np.cov(matrix_meaned, rowvar=False)

# Paso 3: Calcular los valores y vectores propios
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Paso 4: Filtrar valores propios negativos o muy cercanos a cero
eigenvalues = eigenvalues[eigenvalues > 1e-10]  # Filtramos los valores muy pequeños o negativos

# Ordenar los valores propios en orden descendente
sorted_eigenvalues = np.sort(eigenvalues)[::-1]

In [ ]:
sorted_eigenvalues

In [ ]:
# Paso 5: Graficar los valores propios en un gráfico de barras verticales (bar)
plt.figure(figsize=(10, 8))
plt.bar(range(len(sorted_eigenvalues)), sorted_eigenvalues, color='blue')
plt.xlabel('Component Number')
plt.ylabel('Eigenvalue')
plt.title('PCA Eigenvalues Ordered by Importance')

plt.show()

In [ ]:
pc = PCA()
pc.fit(matrix)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Principal Component Analysis')
plt.plot(pc.explained_variance_ratio_)
print(pc.explained_variance_ratio_)
plt.legend(['Explained Variance'])
plt.xlabel('N Components')
plt.ylabel('Explained Variance Ratio')
plt.show()

In [ ]:
pc = PCA(n_components=1)
new_matrix = pc.fit_transform(matrix)
print("Original matrix shape:", matrix.shape)
print("New matrix shape after PCA:", new_matrix.shape)

In [ ]:
# Paso 2: Graficar la varianza explicada por cada componente principal
plt.figure(figsize=(10, 5))
plt.bar(range(1, len(pc.explained_variance_ratio_) + 1), pc.explained_variance_ratio_, color='blue')
print(pc.explained_variance_ratio_)
plt.title('Explained Variance Ratio by Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

## CNN

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ReduceMatrixModel(nn.Module):
    def __init__(self):
        super(ReduceMatrixModel, self).__init__()
        
        # Capas convolucionales para reducir de (1, 2537, 2172) a (256, 8, 6)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2, padding=1)   # -> (batch_size, 32, 1269, 1086)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)                           # -> (batch_size, 64, 635, 543)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)                          # -> (batch_size, 128, 318, 272)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 159, 136)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 80, 68)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 40, 34)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 20, 17)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 10, 9)
        self.conv9 = nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1)                         # -> (batch_size, 256, 5, 5)

        # Capas lineales para reducir de 256*5*5 a 512*768
        self.fc1 = nn.Linear(256 * 5 * 5, 512 * 768)  # -> (batch_size, 512*768)
        
    def forward(self, x):
        # Aplicar convoluciones
        x = F.relu(self.conv1(x))  # -> (batch_size, 32, 1269, 1086)
        x = F.relu(self.conv2(x))  # -> (batch_size, 64, 635, 543)
        x = F.relu(self.conv3(x))  # -> (batch_size, 128, 318, 272)
        x = F.relu(self.conv4(x))  # -> (batch_size, 256, 159, 136)
        x = F.relu(self.conv5(x))  # -> (batch_size, 256, 80, 68)
        x = F.relu(self.conv6(x))  # -> (batch_size, 256, 40, 34)
        x = F.relu(self.conv7(x))  # -> (batch_size, 256, 20, 17)
        x = F.relu(self.conv8(x))  # -> (batch_size, 256, 10, 9)
        x = F.relu(self.conv9(x))  # -> (batch_size, 256, 5, 5)
        
        # Aplanar el tensor para pasar por la capa lineal
        x = x.view(x.size(0), -1)  # -> (batch_size, 256*5*5)
        
        # Pasar por la capa lineal
        x = self.fc1(x)  # -> (batch_size, 512*768)
        
        # Reorganizar el tensor en la forma deseada (batch_size, 512, 768)
        x = x.view(x.size(0), 512, 768)
        
        return x

# Instanciar el modelo y moverlo al dispositivo
reduce = ReduceMatrixModel().to(device)
reduce = reduce.half()


In [8]:
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import bert

In [9]:
vocab_size = tokenizer.vocab_size
class BERT_Arch(nn.Module):
    def __init__(self, bert, vocab_size, max_len=512):
        super(BERT_Arch, self).__init__()
        
        self.bert = bert 
        self.reduce = reduce
        self.vocab_size = vocab_size
        self.max_len = max_len
        
        # Dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # Dense layer (Output layer)
        self.fc = nn.Linear(bert.config.hidden_size, vocab_size)

    def forward(self, seq_input: torch.Tensor):
        # Aplicar reducción de dimensionalidad
        seq_input = seq_input.unsqueeze(1)
        print(seq_input.shape)
        seq_input = self.reduce(seq_input)  # (batch_size, 512)
        print(seq_input.shape)
        # Crear máscara de atención
        attention_mask = torch.ones(seq_input.shape[0], seq_input.shape[1], seq_input.shape[2]).to(seq_input.device).to(torch.half)
        print(seq_input.shape == attention_mask.shape)
        print(seq_input.shape, attention_mask.shape, seq_input.dtype, attention_mask.dtype)
        print(seq_input[0][0])
        outputs = self.bert(inputs_embeds=seq_input, attention_mask=attention_mask, return_dict=True)
        print("salio")
        x = outputs.last_hidden_state  # Usar el último estado oculto

        # Pasar por la capa densa
        logits = self.fc(x)  # logits tiene forma (batch_size, 512, vocab_size)

        return logits
    
    def generate(self, input_ids, max_length):
        self.eval()
        generated_tokens = []

        # Inicializar el estado de entrada para generación
        for _ in range(max_length):
            outputs = self.forward(input_ids)
            next_token = torch.argmax(outputs[:, -1, :], dim=-1)  # Obtener el token más probable
            generated_tokens.append(next_token.item())
            
            # Actualizar input_ids para el siguiente token
            input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=1)

            # Salir si se alcanza el token de fin de secuencia
            if next_token.item() == tokenizer.eos_token_id:
                break
        return generated_tokens

# Inicializar el modelo con BERT y vocab_size
model = BERT_Arch(bert, vocab_size)

# Mover el modelo a la GPU
model = model.to(device)

In [ ]:
outputs = model(torch.tensor([matrix]).to(device))

In [ ]:
print(outputs.shape)

In [16]:
predicted_tokens = torch.argmax(outputs, dim=-1)  # (batch_size, seq_length)
predicted_tokens = predicted_tokens[0].tolist()
decoded_text = tokenizer.decode(predicted_tokens, skip_special_tokens=True)

In [ ]:
print("Texto generado:", decoded_text)
